In [1]:
import jax
import os
import jax.numpy as np
import jax_cosmo as jc
import numpy as onp
from desy1 import theory_cov, get_params_vec, get_data


In [2]:

fid_cosmo = jc.Cosmology(sigma8=0.801,
                          Omega_c=0.2545,
                          Omega_b=0.0485,
                          h=0.682,
                          n_s=0.971,
                          w0=-1., Omega_k=0., wa=0.)

fid_params  = get_params_vec(fid_cosmo, 
                                          [0., 0., 0., 0.],
                                          [0., 0., 0., 0.],
                                          [0.5, 0.],
                                          [1.2, 1.4, 1.6, 1.8, 2.0])
nz_source, nz_lens = get_data()

In [3]:
neff_s = [1.47, 1.46, 1.50, 0.73]

nzs_s = [jc.redshift.kde_nz(nz_source['Z_MID'].astype('float32'),
                            nz_source['BIN%d'%i].astype('float32'), 
                            bw=0.01,
                            gals_per_arcmin2=neff_s[i-1])
           for i in range(1,5)]
nzs_l = [jc.redshift.kde_nz(nz_lens['Z_MID'].astype('float32'),
                              nz_lens['BIN%d'%i].astype('float32'), bw=0.01)
           for i in range(1,6)]    


In [4]:
# Define some ell range
ell = np.logspace(1, 3)
args = [nzs_s, nzs_l, ell]
covmat = theory_cov(fid_params, *args)
covmat = onp.array(covmat)
#print(covmat)
onp.save("covmat.npy", covmat)

I LIKE BIG BUTTS AND I CANNOT LIE


TracerArrayConversionError: The numpy.ndarray conversion method __array__() was called on the JAX Tracer object Traced<ShapedArray(float64[2025,50,50])>with<DynamicJaxprTrace(level=0/1)>
While tracing the function theory_cov at /jax-cosmo-paper/notebooks/desy1.py:120 for jit, this concrete value was not available in Python because it depends on the values of the arguments 'p', 'nzs_s', 'nzs_l', and 'ell'.
See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.TracerArrayConversionError

In [ ]:
covmat = onp.load("covmat.npy")
#print(covmat)
print(covmat.shape)

In [ ]:
#cov_mat # shape = (n_cls*n_cls, n_ell)
n_cls = 45
n_ell = 50
c = onp.zeros((n_cls * n_ell, n_cls * n_ell))
#onp.set_printoptions(threshold=np.inf)
#print(onp.nonzero(covmat)[2]-onp.nonzero(covmat)[1])
# for i in range(n_cls):
#     for j in range(n_cls):
#         s1 = i * n_ell
#         s2 = j * n_ell
#         for k in range(n_ell):
#             c[s1 + k, s2 + k] = covmat[i + n_cls*j, k]
# covmat = c

# C = []
# for i in covmat:
#     for j in range(50):
#         C.append(i[j][j])
        
# Cov = onp.array(C)
#print(Cov.reshape((50,45,45)))

for i in range(n_cls):
    for j in range(n_cls):
        s1 = i * n_ell
        s2 = j * n_ell

        c[s1:s1+n_ell, s2:s2+n_ell] = covmat[i + n_cls*j]
cov = c
print(cov.shape)
print(cov)
    


In [ ]:
def check_symmetric(a, tol=1e-30):
    return np.all(np.abs(a-a.T) < tol)